In [3]:
import socket, os, subprocess, json, ssl, threading, logging, time, selectors
class Server:
    def __init__(self):
        self.selector =  selectors.DefaultSelector()
        self.databaseHolder = self.read()
        self.databaseWriter = lambda x : self.write(x)
        self.executeBash = lambda x: subprocess.Popen(x, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT).communicate()[0]
        self.encoder = lambda x: x.encode()
        self.decoder = lambda x: x.decode()
        self.readMessage = lambda x: self.decoder(x.recv()) # x represents socket
        self.writeMessage = lambda x, y: x.sendall(self.encoder(y)) # x represents socket, y represents raw message
        #self.NoneGUIOperator = lambda x: self.writeMessage(x, input()) # Terminal Operatior for None GUI use case
        self.lock = lambda : threading.Lock()
        self.context = self.initContext()
        self.clients = {}
        self.wakeServer()
    def initContext(self) -> ssl.SSLContext:
        context = ssl.SSLContext(ssl.PROTOCOL_TLS_SERVER)
        context.load_cert_chain('Cert/domain.crt', 'Cert/domain.key')
        return context
    def wakeServer(self):
        client = 0
        try:
            with socket.socket(socket.AF_INET, socket.SOCK_STREAM, 0) as sock:
                sock.bind(('192.168.1.102', 15000)) # needed to be configured for spesific host machine, can be automized wit using executeBash command
                sock.listen()
                ssock = self.context.wrap_socket(sock, server_side=True)
                while True:
                    print(client)
                    self.clients[client] = [0, 0]
                    #self.clients[client][1] = ssock
                    self.clients[client][0], addr = ssock.accept()
                    print('connection from: ', addr)
                    #self.clients[client] = (conn, ssock)
                    threading.Thread(target=self.clientHandler, args=(self.clients[client][0], client)).start()
                    print('Proceed')
                    client += 1
                    print(client)
        except Exception as e:
            print(e)
    
    def is_socket_closed(self, sock: socket.socket) -> bool:
        try:
            # this will try to read bytes without blocking and also without removing them from buffer (peek only)
            data = sock.recv(16, socket.MSG_DONTWAIT | socket.MSG_PEEK)
            if len(data) == 0:
                return True
        except BlockingIOError:
            return False  # socket is open and reading from it would block
        except ConnectionResetError:
            return True  # socket was closed for some other reason
        except Exception as e:
            print("unexpected exception when checking if a socket is closed")
            print(e)
            return False
        return False
    def loginNonGUI(self, conn, connID):
        #self.writeMessage(conn, 'Please enter your School ID: ')
        schoolID = self.readMessage(conn)
        #self.writeMessage(conn, 'please enter your password')
        password = self.readMessage(conn)
        if schoolID in self.databaseHolder.keys():
            if password == self.databaseHolder[schoolID]['Password']:
                self.databaseHolder[schoolID]['Online'] = True
                self.clients[schoolID] = self.clients.pop(connID)
                self.writeMessage(conn, 'Server,True"')
                return 1, schoolID
            
        return 0, schoolID
    def signUp(self, conn):
        #self.writeMessage(conn, 'Please enter your School ID: ')
        mail = self.readMessage(conn)
        self.databaseHolder[mail] = {}
        for key in ('Name', 'Surname', 'Mail Address', 'Password'):
            self.writeMessage(conn, 'Please enter your {}: '.format(key))
            self.databaseHolder[mail][key] = self.readMessage(conn)
        self.databaseHolder[mail]['Online'] = False
        self.write(self.databaseHolder)
    def clientHandler(self, conn: socket.socket, client: int):
        recThread, writeThread = 1, 1
        #self.writeMessage(conn, 'Hello, please enter login if you have an account, otherwise, type sign up')
        print('proceed')
        msg = self.readMessage(conn)
        print('Someone Connected')
        if msg.lower() == 'sign up':
            self.signUp(conn)
           # self.writeMessage(conn, 'forwarding to Login phase')
        loginCheck, ID = self.loginNonGUI(conn, client)
        if loginCheck:
            #self.writeMessage(conn, 'enter student number of person whom message will be directed, write with comma seperation for mutliple forwarding, ex. 117200033,114200045,117600023: ')
            directions = self.readMessage(conn).split(',')
            for direction in directions:
                on = 1
                while not (self.is_socket_closed(conn)):
                    print(self.databaseHolder[direction]['Online'] == False)
                    if  self.databaseHolder[direction]['Online']:
                        on = 1
                        if not self.is_socket_closed(self.clients[direction][0]):
                            # self.writeMessage(conn, '')
                            msg = self.readMessage(conn)
                            msg = ID + ': ' + msg
                            
                            self.writeMessage(self.clients[direction][0], msg)
                        else:
                            self.clients[direction][0].close()
                    else:
                         if on:
                                self.writeMessage(conn, 'Student you want to message is currently offline, please try again later, currently active students are: {}'.format(', '.join([_id for _id in self.databaseHolder.keys() if self.databaseHolder[_id]['Online']])))
                                self.writeMessage(conn, 'enter student number of person whom message will be directed: ')
                                direction = self.readMessage(conn)
                self.clients[ID][0].close()
                self.databaseHolder[ID]['Online'] = False
        else:
            self.writeMessage(conn, 'Creds are wrong: ')
            self.clientHandler(self.clients[client][0], client)
    def read(self):
        with open('Database/loginCreds.json', 'r') as f: 
            return json.load(f)
    def write(self, data):
        try: 
            with open('Database/loginCreds.json', 'w') as f:
                json.dump(data, f)
                return 1
        except Exception as e:
            print(e)
            return 0

        
        

In [ ]:
Server()

0
connection from:  ('192.168.1.102', 59336)
proceed
Someone ConnectedProceed

1
1


Exception in thread Thread-4:
Traceback (most recent call last):
  File "/usr/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.8/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-3-8a2970bcfc2f>", line 95, in clientHandler
KeyError: ''


unexpected exception when checking if a socket is closed
non-zero flags not allowed in calls to recv() on <class 'ssl.SSLSocket'>
connection from:  ('192.168.1.102', 59350)
proceed
Proceed
2
2
Someone Connected
connection from:  ('192.168.1.102', 59354)
proceed
Someone Connected
Proceed
3
3


Exception in thread Thread-6:
Traceback (most recent call last):
  File "/usr/lib/python3.8/threading.py", line 932, in _bootstrap_inner
Exception in thread Thread-5:
Traceback (most recent call last):
  File "/usr/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.8/threading.py", line 870, in run
    self.run()
  File "/usr/lib/python3.8/threading.py", line 870, in run
        self._target(*self._args, **self._kwargs)self._target(*self._args, **self._kwargs)
  File "<ipython-input-3-8a2970bcfc2f>", line 95, in clientHandler

  File "<ipython-input-3-8a2970bcfc2f>", line 95, in clientHandler
KeyErrorKeyError: ''
: ''


unexpected exception when checking if a socket is closed
non-zero flags not allowed in calls to recv() on <class 'ssl.SSLSocket'>
unexpected exception when checking if a socket is closed
non-zero flags not allowed in calls to recv() on <class 'ssl.SSLSocket'>


In [1]:
'aa' - 'a'

TypeError: unsupported operand type(s) for -: 'str' and 'str'

In [5]:
','.join('aa,asdasfdasdgsdas,sd'.split(',')[1:])

'asdasfdasdgsdas,sd'